<h1> Testing PASTIS in imaging mode </h1>

## -- ATLAST aperture --

Here we're testing the module `image_pastis.py` which is the version of PASTIS that still generates images. Since this is a module, it's a tiny bit harder to test, so I'm basically just going through the code step by step.

In [ ]:
import os
import numpy as np
from astropy.io import fits
import astropy.units as u
import poppy.zernike as zern
import poppy.matrixDFT as mft
import poppy
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from hcipy import *

os.chdir('../../pastis/')
from config import CONFIG_INI
import util_pastis as util

## Setup and single segment aberration

Since this is a module, its function will be called with input parameters, which I will define separetaly here to be able to use them.

Taken from `calibration.py`.

In [ ]:
# Define the aberration coeffitients "coef"
which_tel = CONFIG_INI.get('telescope', 'name')
nb_seg = CONFIG_INI.getint(which_tel, 'nb_subapertures')
zern_max = CONFIG_INI.getint('zernikes', 'max_zern')

nm_aber = CONFIG_INI.getfloat('calibration', 'single_aberration') * u.nm   # [nm] amplitude of aberration
zern_number = CONFIG_INI.getint('calibration', 'zernike')                  # Which (Noll) Zernike we are calibrating for

### What segmend are we aberrating? ###
segnum = 0   # segment 1 --> i=0, seg 2 --> i=1, etc.
### ------------------------------- ###

# Create arrays to hold Zernike aberration coefficients
Aber_Noll = np.zeros([nb_seg, zern_max])           # The Zernikes here will be filled in the Noll order.

# Feed the aberration nm_aber into the array position
# that corresponds to the correct Zernike, but only on segment i
Aber_Noll[segnum, zern_number-1] = nm_aber.value            # Noll version - in input units directly!

# Vector of aberration coefficients takes all segments, but only for the Zernike we currently work with
coef = Aber_Noll[:,zern_number-1]

# Make sure the aberration coefficients have correct units
coef *= u.nm

# Define the (Noll) zernike number
zernike_pol = zern_number

# We're not calibrating
cali=False

print('Working on segment: {}'.format(segnum+1))
print('coef: {}'.format(coef))
print('Aberration: {}'.format(nm_aber))
print('Zernike (Noll): {}'.format(util.zernike_name(zern_number, framework='Noll')))
print('Zernike number (Noll): {}'.format(zernike_pol))

Now we'll start with the actual code in the module `image_pastis.py`.

In [ ]:
#-# Parameters
dataDir = os.path.join(CONFIG_INI.get('local', 'local_data_path'), 'active')
nb_seg = CONFIG_INI.getint(which_tel, 'nb_subapertures')
tel_size_m = CONFIG_INI.getfloat(which_tel, 'diameter') * u.m
real_size_seg = CONFIG_INI.getfloat(which_tel, 'flat_to_flat')   # size in meters of an individual segment flatl to flat
size_seg = CONFIG_INI.getint('numerical', 'size_seg')              # pixel size of an individual segment tip to tip
wvln = CONFIG_INI.getint(which_tel, 'lambda') * u.nm
inner_wa = CONFIG_INI.getint(which_tel, 'IWA')
outer_wa = CONFIG_INI.getint(which_tel, 'OWA')
tel_size_px = CONFIG_INI.getint('numerical', 'tel_size_px')        # pupil diameter of telescope in pixels
im_size_pastis = CONFIG_INI.getint('numerical', 'im_size_px_pastis')   # image array size in px
im_size_e2e = CONFIG_INI.getint('numerical', 'im_size_px_webbpsf')
sampling = CONFIG_INI.getfloat('numerical', 'sampling')            # sampling
size_px_tel = tel_size_m / tel_size_px                             # size of one pixel in pupil plane in m
px_sq_to_rad = (size_px_tel * np.pi / tel_size_m) * u.rad
zern_max = CONFIG_INI.getint('zernikes', 'max_zern')

#############
# -) for keeping image size consistency with WebbPSF, use im_size_pastis/sampling
# -) for actually being able to see something, make it smaller with a random number of r = sz * lambda/D,
#    which translates to a focal plane size of 2*sz*sampling
sz = 15 # lambda/D
sampling = 4
#############
    
# Create Zernike mode object for easier handling
zern_mode = util.ZernikeMode(zernike_pol)

#-# Mean subtraction for piston
if zernike_pol == 1:
    coef -= np.mean(coef)

#-# Generic segment shapes
# Load pupil from file
pupil = fits.getdata(os.path.join(dataDir, 'segmentation', 'pupil.fits'))
pup_im = np.copy(pupil)

print('Pupil shape:', pupil.shape)

plt.imshow(pupil, origin='lower')
plt.show()

### Creating a mini segment

At this point, you have to make sure the pixel size  **size_seg** of your individual segment is correct, and this will be different depending on the pixel size of your total pupil.

With that, we create a mini segment.

In [ ]:
# Setting up pupil and focal grid and propagator
pupil_grid = make_pupil_grid(dims=tel_size_px, diameter=1)
focal_grid = make_focal_grid(pupil_grid, sampling, sz)    # pixels per lambda/D, lambda/D radius of total image
prop = FraunhoferPropagator(pupil_grid, focal_grid)       # this is without a coronagraph

mini_seg_real = hexagonal_aperture(circum_diameter=1, angle=np.pi/2)
mini_seg = evaluate_supersampled(mini_seg_real, pupil_grid, 4)  # the supersampling number doesn't really matter in context with the other numbers
mini_seg_2d = mini_seg.shaped    # make it a 2D array

# Redefine size_seg if using HCIPy
size_seg = mini_seg_2d.shape[0]

print("HCIPy array size: {}".format(mini_seg_2d.shape))

imshow_field(mini_seg)
plt.title("mini segment from HCIPy")
plt.show()

We managed to cut the array to a square where the mini-segment is just about touching the array edges. The size of this array is the size of our mini-segment and that's a number that we have to enter into the configfile. Enter the pixel size of the mini-segment array into the configfile section: **[numerical] --> size_seg**

### Generating the dark hole

In [ ]:
#-# Generate a dark hole

# This can probably be made more straight forward
dh_area = util.create_dark_hole(pup_im, inner_wa, outer_wa, sampling)
print('DH array shape:', dh_area.shape)

boxsize = sz*sampling
dh_sz = util.zoom_cen(dh_area, boxsize)
    
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(dh_area, origin='lower')
plt.title('Dark hole')
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(dh_area, boxsize), origin='lower')
plt.title('Dark hole zoomed in')
plt.show()

### Importing the matrix

In [ ]:
#-# Import information form aperture generation script
Projection_Matrix = fits.getdata(os.path.join(dataDir, 'segmentation', 'Projection_Matrix.fits'))
vec_list = fits.getdata(os.path.join(dataDir, 'segmentation', 'vec_list.fits'))   # in pixels
NR_pairs_list = fits.getdata(os.path.join(dataDir, 'segmentation', 'NR_pairs_list_int.fits'))
NR_pairs_nb = NR_pairs_list.shape[0]
    
plt.imshow(Projection_Matrix[:,:,0], origin='lower')
plt.title('Projection matrix displaying NRPs')
plt.colorbar()
plt.show()
    
print('Non-redundant pairs (' + str(NR_pairs_nb) + '):')
print(NR_pairs_list)

### Calculating the (uncalibrated) analytical image

We don't have calibration coefficients yet, so we skip the "if cali:" part.

Move on to the calculation of eq. 13 in the paper Leboulleux et al. 2018 that calculates the image intensity of the analytical images:

$$I(u) = ||\hat{Z_l}(u)||^2  \Bigg[ \sum_{k=1}^{n_{seg}} a^2_{k,l} + 2 \sum_{q=1}^{n_{NRP}} A_q cos(b_q \cdot u) \Bigg] $$

#### Generic coefficients

$A_q$... *generic coefficients*

$$A_q = \sum_{(i,j)} a_{i,l} a_{j,l}$$

In [ ]:
#-# Generic coefficients
generic_coef = np.zeros(NR_pairs_nb) * u.nm * u.nm   # coefficients in front of the non redundant pairs,
                                                     # the A_q in eq. 13 in Leboulleux et al. 2018

for q in range(NR_pairs_nb):
    for i in range(nb_seg):
        for j in range(i+1, nb_seg):
            if Projection_Matrix[i, j, 0] == q+1:
                print('q:', q, 'i:', i, 'j:', j)
                generic_coef[q] += coef[i] * coef[j]
                print('ci:', coef[i], 'cj:', coef[j])

In [ ]:
print('Generic coefficients:')
print(generic_coef)

#### Sum over the aberration coefficients and sum over the cosines

$cos(b_q \cdot u)$... `cos_u_mat`

Sum over $a^2_{k,l}$... `sum1`

In [ ]:
#-# Constant sum and cosine sum - calculating eq. 13 from Leboulleux et al. 2018
i_line = np.linspace(-(2*sz*sampling)/2., (2*sz*sampling)/2., (2*sz*sampling))
tab_i, tab_j = np.meshgrid(i_line, i_line)
cos_u_mat = np.zeros((int((2*sz*sampling)), int((2*sz*sampling)), NR_pairs_nb))
    
# The -1 with each NR_pairs_list is because the segment names are saved starting from 1, but Python starts
# its indexing at zero, so we have to make it start at zero here too.
for q in range(NR_pairs_nb):
    cos_u_mat[:,:,q] = np.cos(px_sq_to_rad * (vec_list[NR_pairs_list[q,0]-1, NR_pairs_list[q,1]-1, 0] * tab_i) + 
                              px_sq_to_rad * (vec_list[NR_pairs_list[q,0]-1, NR_pairs_list[q,1]-1, 1] * tab_j)) * u.dimensionless_unscaled

sum1 = np.sum(coef**2)   # sum of all a_{k,l} in eq. 13 - this works only for single Zernikes (l fixed), because np.sum would sum over l too, which would be wrong.
    
print('cos:', cos_u_mat)
print('sum1:', sum1)

$\Bigg[ \sum_{k=1}^{n_{seg}} a^2_{k,l} + 2 \sum_{q=1}^{n_{NRP}} A_q cos(b_q \cdot u) \Bigg]$ `= sum2 + generic_coef[q] * cos_u_mat[:,:,q]`

In [ ]:
sum2 = np.zeros((int(2*sz*sampling), int(2*sz*sampling))) * u.nm * u.nm
for q in range(NR_pairs_nb):
    sum2 = sum2 + generic_coef[q] * cos_u_mat[:,:,q]
    
print('sum2:', sum2)

#### The global envelope from the mini segment Zernike

In [ ]:
#-# Local Zernike
# Generate a basis of Zernikes with the mini segment being the support
#isolated_zerns = zern.hexike_basis(nterms=zern_max, npix=size_seg, rho=None, theta=None, vertical=False, outside=0.0)
isolated_zerns = make_zernike_basis(num_modes=zern_max, D=1, grid=pupil_grid, radial_cutoff=True)
isolated_zerns_nocut = make_zernike_basis(num_modes=zern_max, D=1, grid=pupil_grid, radial_cutoff=False)

In [ ]:
print(isolated_zerns[1].shaped.shape)
print(type(isolated_zerns[1].shaped))

for i in range(zern_max):
    plt.subplot(3, 4, i+1)
    plt.imshow(isolated_zerns[i].shaped)
plt.show()

In [ ]:
# Also show the ones that are not cut by circle
for i in range(zern_max):
    plt.subplot(3, 4, i+1)
    plt.imshow(isolated_zerns_nocut[i].shaped)
plt.show()

In [ ]:
# Display the wavefront with radial cut
imshow_field(isolated_zerns[1])
plt.show()

In [ ]:
# Make a propagated image from the Zernike wavefront and display different
# attributes of the resulting wavefront.
wf1 = Wavefront(isolated_zerns[0])
im = prop.forward(wf1)

plt.figure(figsize=(20, 20))
plt.subplot(2, 2, 1)
imshow_field(np.log10(im.amplitude))
plt.title("Amplitude")
plt.colorbar()
plt.subplot(2, 2, 2)
imshow_field(im.phase)
plt.title("Phase")
plt.colorbar()
plt.subplot(2, 2, 3)
imshow_field(im.real)
plt.title("Real")
plt.colorbar()
plt.subplot(2, 2, 4)
imshow_field(im.imag)
plt.title("Imag")
plt.colorbar()
plt.show()

In [ ]:
# Display intensity either by directly using the intensity attribute,
# or by equivalently getting it from the electric field directly.

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title("Intensity attribute")
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(np.log10(np.abs(im.electric_field**2)))
plt.title("Intensity from E-field")
plt.colorbar()
plt.show()

# Obviously, they should be the same.

In [ ]:
# Create Wavefront (E-field) of aperture and Zernike mode and display phase
plt.figure(figsize=(20, 15))
for i in range(zern_max):
    wf_mode = Wavefront(mini_seg * np.exp(1j * isolated_zerns[i]))
    plt.subplot(3, 4, i+1)
    imshow_field(wf_mode.phase, mask=mini_seg, cmap='RdBu') # mask by mini_seg in order to cut phase outside of aperture out
plt.show()

In [ ]:
# Create Wavefront (E-field) of aperture and Zernike mode and display phase
# this time for non-radially cut Zernikes
plt.figure(figsize=(20, 15))
for i in range(zern_max):
    wf_mode = Wavefront(mini_seg * np.exp(1j * isolated_zerns_nocut[i]))
    plt.subplot(3, 4, i+1)
    imshow_field(wf_mode.phase, mask=mini_seg, cmap='RdBu')
    # This is equivalent to doing imshow_field(isolated_zerns_nocut[i]) in line below
    #imshow_field(isolated_zerns_nocut[i], mask=mini_seg, cmap='RdBu')
plt.show()

In [ ]:
# Calculate the Zernike that is currently being used and put it on one single subaperture,
# the result is the E-field Zer.
# Apply the currently used Zernike to the mini-segment.
#zernike_pol = 1
Zer_wf = Wavefront(mini_seg * np.exp(1j * isolated_zerns_nocut[zernike_pol-1]))
    
print(type(Zer_wf))
print('Workingon Zernike number: {}'.format(zernike_pol))

imshow_field(Zer_wf.phase, mask=mini_seg, cmap='RdBu')
plt.title("Pupil phase with Zernike")
plt.show()

# Fourier Transform of the Zernike - the global envelope

Note how we are **NOT** propagating a full E-field of the form $E = A\ e^{i\phi}$, but $\phi$ only (cut down to $A$, which is here the mini-segment aperture). We still have to make a `Wavefront` out of it, otherwise we can't use the propagator.

In [ ]:
Zer = Wavefront(mini_seg * isolated_zerns_nocut[zernike_pol-1])
ft_hcipy = prop(Zer)

imshow_field(np.log(ft_hcipy.intensity))
plt.title("Zernike envelope")
plt.show()

So the propagated Zernike mode, meaning the propagated Zernike envelope, is `ft_hcipy`. What kind of object is this though? And is it appropriate to use its intensity as the result we need?

In [ ]:
print(type(ft_hcipy))
print(ft_hcipy.intensity.shaped.shape)

In [ ]:
print("Shape of ft_mini: {}".format(ft_hcipy.intensity.grid.shape))

#### What do those envelopes look like for other Zernikes?

Also check the global envelopes from the other Zernikes on the mini-segment.

In [ ]:
# Propagate all Zernikes individually to image plane
mini_ft = []
for i in range(zern_max):
    wf_mode = Wavefront(mini_seg * isolated_zerns_nocut[i])
    ft_klein = prop(wf_mode)
    mini_ft.append(ft_klein)
mini_ft = np.array(mini_ft)

In [ ]:
# Display the envelopes
plt.figure(figsize=(15, 20))
plt.suptitle('Different Zernike envelopes')
for i in range(zern_max):
    plt.subplot(4, 3, i+1)
    imshow_field(np.log10(mini_ft[i].intensity))
    plt.title('Noll Zernike: ' + str(i+1))
plt.show()

They correspond to what Lucie has in her paper, which is pretty cool. Here, I even have control over the sampling because I am not bound to WebbPSF.

#### Putting things together and calculating the full image

Moving on. Calculating the full $I(u)$ now.

$$I(u) = ||\hat{Z_l}(u)||^2  \Bigg[ \sum_{k=1}^{n_{seg}} a^2_{k,l} + 2 \sum_{q=1}^{n_{NRP}} A_q cos(b_q \cdot u) \Bigg] $$


In [ ]:
sum2.value

In [ ]:
print('Aberration coefficients on segments:\n{}'.format(coef))

One of the coefficients should be way larger than the others because that is the segment we put the single aberration on. The other ones are not zero because we subtracted the mean out of the `coef` array.

In [ ]:
#-# Final image
# Generating the final image that will get passed on to the outer scope, I(u) in eq. 13
print('zernike_pol: {}'.format(zernike_pol))
final_im = ft_hcipy.intensity.shaped * (sum1.value + 2. * sum2.value)
# it doesn't matter if I use ft_hcipy.intensity or np.abs(ft_hcipy.electric_field)**2 here

plt.figure(figsize=(10,10))
plt.imshow(final_im, norm=LogNorm(), origin='lower')
plt.title('Final image, single aberrated segment: {}'.format(segnum+1))
plt.show()



#### Extracting the dark hole

In [ ]:
# PASTIS is only valid inside the dark hole.
tot_dh_im_size = sampling*(outer_wa+3)                     # zoom box must be big enough to capture entire DH

# These two will not be used or need to be adapted in the future
final_im_zoom = util.zoom_cen(final_im, tot_dh_im_size)
dh_area_zoom = util.zoom_cen(dh_area, tot_dh_im_size)

#dh_psf = dh_area_zoom * intensity_zoom
dh_psf = dh_sz * final_im

# Display dark hole and inner part of image next ot each other, on the same scale
plt.figure(figsize=(19,10))
plt.subplot(1, 2, 1)
plt.imshow(dh_psf, norm=LogNorm(), origin='lower')
plt.title('Final image - dark hole only')
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(final_im, tot_dh_im_size), norm=LogNorm(), origin='lower')
plt.title('Without mask for comparison, not quite same image size')
plt.show()

Crop out the DH for the Zernike envelopes.

In [ ]:
dh_sz.shape

In [ ]:
mini_im = np.zeros_like(mini_ft)
for i in range(len(mini_ft)):
    mini_im[i] = mini_ft[i].intensity     # don't forget that mini_ft is a list of wavefront objects
mini_dh_stack = []

plt.figure(figsize=(15, 25))
plt.suptitle('DH area of individual Zernike envelopes')
for i in range(mini_ft.shape[0]):
    #mini_zoom = util.zoom_cen(mini_im[i], tot_dh_im_size)
    mini_dh = dh_sz * mini_im[i].shaped
    mini_dh_stack.append(mini_dh)
    
    plt.subplot(6, 2, i+1)
    plt.imshow(np.abs(mini_dh), norm=LogNorm(), origin='lower')
    plt.title('Noll Zernike: ' + str(i+1))
    
plt.show()

### Aberrating pairs of segments

We now want to explore what the final analytical image looks like when we aberrate two segments at a time, with the same aberration.

In [ ]:
# Decide which two segments you want to aberrate
segnum1 = 19     # Which segments are we aberrating - I number them starting with 1
segnum2 = 28

segnum_array = np.array([segnum1, segnum2])

zern_pair = 2  # Which Noll Zernike are we putting on the segments.

print('Aberrated segments:', segnum_array)
print('Noll Zernike used:', zern_pair)

# Create aberration vector
Aber_Noll = np.zeros([nb_seg, zern_max])
print('nm_aber: {}'.format(nm_aber))

# Fill aberration array
for i, nseg in enumerate(segnum_array):
    Aber_Noll[nseg-1, zern_pair-1] = nm_aber.value   # fill only the index for current Zernike, on segment i - in input units
    
# Define the aberration coefficient vector
coef = Aber_Noll[:, zern_pair-1]
coef *= u.nm
print('coef:', coef)

In [ ]:
#-# Generic coefficients
generic_coef = np.zeros(NR_pairs_nb) * u.nm * u.nm   # coefficients in front of the non redundant pairs,
                                                     # the A_q in eq. 13 in Leboulleux et al. 2018

for q in range(NR_pairs_nb):
    for i in range(nb_seg):
        for j in range(i+1, nb_seg):
            if Projection_Matrix[i, j, 0] == q+1:
                generic_coef[q] += coef[i] * coef[j]
                
print('Generic coefficients:')
print(generic_coef)

#-# Constant sum and cosine sum - calculating eq. 13 from Leboulleux et al. 2018
i_line = np.linspace(-(2*sz*sampling)/2., (2*sz*sampling)/2., (2*sz*sampling))
tab_i, tab_j = np.meshgrid(i_line, i_line)   # these are arrys for the image plane coordinate u
cos_u_mat = np.zeros((int((2*sz*sampling)), int((2*sz*sampling)), NR_pairs_nb))
    
# The -1 with each NR_pairs_list is because the segment names are saved starting from 1, but Python starts
# its indexing at zero, so we have to make it start at zero here too.
for q in range(NR_pairs_nb):
    cos_u_mat[:,:,q] = np.cos(px_sq_to_rad * (vec_list[NR_pairs_list[q,0]-1, NR_pairs_list[q,1]-1, 0] * tab_i) + 
                              px_sq_to_rad * (vec_list[NR_pairs_list[q,0]-1, NR_pairs_list[q,1]-1, 1] * tab_j)) * u.dimensionless_unscaled

sum1 = np.sum(coef**2)   # sum of all a_{k,l} in eq. 13 - this works only for single Zernikes (l fixed), because np.sum would sum over l too, which would be wrong.
    
#print('cos:', cos_u_mat)
#print('sum1:', sum1)

sum2 = np.zeros((int((2*sz*sampling)), int((2*sz*sampling)))) * u.nm * u.nm
for q in range(NR_pairs_nb):
    sum2 = sum2 + generic_coef[q] * cos_u_mat[:,:,q]
    
#print('sum2:', sum2)

In [ ]:
# Calculate the Zernike that is currently being used and put it on one single subaperture, the result is Zer
# Apply the currently used Zernike to the mini-segment.
Zer = Wavefront(mini_seg * isolated_zerns_nocut[zern_pair-1])

imshow_field(Zer.intensity, mask=mini_seg, cmap='RdBu')
plt.title('Zernike mode on mini segment')
plt.show()

In [ ]:
# Fourier Transform of the Zernike - the global envelope
ft_hcipy = prop(Zer)

imshow_field(np.log10(ft_hcipy.intensity))
plt.title("Zernike envelope")
plt.show()

In [ ]:
#-# Final image
# Generating the final image that will get passed on to the outer scope, I(u) in eq. 13
final_im = ft_hcipy.intensity.shaped * (sum1.value + 2. * sum2.value)

plt.figure(figsize=(10,10))
plt.imshow(final_im, norm=LogNorm(), origin='lower')
plt.title('Final image')
plt.show()

#### Saving some of the images

I will save a couple of images down here to be able to display them next to each other:

In [ ]:
# PISTON - Noll 1
#segs_1_2_noll_1 = np.copy(final_im)
#segs_1_10_noll_1 = np.copy(final_im)
#segs_1_24_noll_1 = np.copy(final_im)
#segs_1_7_noll_1 = np.copy(final_im)
#segs_1_19_noll_1 = np.copy(final_im)
#segs_19_28_noll_1 = np.copy(final_im)

# TIP - Noll 2
#segs_1_2_noll_2 = np.copy(final_im)
#segs_1_10_noll_2 = np.copy(final_im)
#segs_1_24_noll_2 = np.copy(final_im)
#segs_1_7_noll_2 = np.copy(final_im)
#segs_1_19_noll_2 = np.copy(final_im)
#segs_19_28_noll_2 = np.copy(final_im)

In [ ]:
# Save them all to fits files
save_dir = '/astro/opticslab1/PASTIS/atlast_data/uncalibrated_analytical_images/2019-04-23-13h-00min'
# util.write_fits(segs_1_2_noll_1, os.path.join(save_dir, 'segs_1_2_noll_1.fits'))
# util.write_fits(segs_1_10_noll_1, os.path.join(save_dir, 'segs_1_10_noll_1.fits'))
# util.write_fits(segs_1_24_noll_1, os.path.join(save_dir, 'segs_1_24_noll_1.fits'))
# util.write_fits(segs_1_7_noll_1, os.path.join(save_dir, 'segs_1_7_noll_1.fits'))
# util.write_fits(segs_1_19_noll_1, os.path.join(save_dir, 'segs_1_19_noll_1.fits'))
# util.write_fits(segs_19_28_noll_1, os.path.join(save_dir, 'segs_19_28_noll_1.fits'))

# util.write_fits(segs_1_2_noll_2, os.path.join(save_dir, 'segs_1_2_noll_2.fits'))
# util.write_fits(segs_1_10_noll_2, os.path.join(save_dir, 'segs_1_10_noll_2.fits'))
# util.write_fits(segs_1_24_noll_2, os.path.join(save_dir, 'segs_1_24_noll_2.fits'))
# util.write_fits(segs_1_7_noll_2, os.path.join(save_dir, 'segs_1_7_noll_2.fits'))
# util.write_fits(segs_1_19_noll_2, os.path.join(save_dir, 'segs_1_19_noll_2.fits'))
# util.write_fits(segs_19_28_noll_2, os.path.join(save_dir, 'segs_19_28_noll_2.fits'))

#### Display and compare the images

I started making PASTIS images from the ATLAST pupil (generated by HCIPy) and below here, I am just showing the general properties of PASTIS images with the ATLAST example, just like in the JWST notebook. I have made a lot more different data with the entrance pupil, which is why I am using those images for the demo below here. This data is stored in: `/astro/opticslab1/PASTIS/atlast_data/uncalibrated_analytical_images/2019-04-23-13h-00min_100nm_piston`.

#### *Fringe orientation and spacing*

Let's start with the image in which we put **piston** on **segments 1 and 2**. When we check in what relation those two segments lie to each other on the (exit!) pupil (open the pupil image file for help), we can see:

1. They like on a vertical offset to each other.
2. They are very close together, the closest two segments on the ATLAST can be.

This means for the image:
1. The fringes are are tilted by the same amount like the connection vector between the two segments, but flipped by 90 degrees.
2. Since the segments are **close together**, the fringes in the Fourier plane, here the final image, will be **wide**.

In [ ]:
# I need to read the images in now
read_dir1 = '/astro/opticslab1/PASTIS/atlast_data/uncalibrated_analytical_images/2019-04-23-13h-00min_100nm_piston'
segs_1_2_noll_1 = fits.getdata(os.path.join(read_dir1, 'segs_1_2_noll_1.fits'))
segs_1_10_noll_1 = fits.getdata(os.path.join(read_dir1, 'segs_1_10_noll_1.fits'))
segs_1_24_noll_1 = fits.getdata(os.path.join(read_dir1, 'segs_1_24_noll_1.fits'))
segs_1_7_noll_1 = fits.getdata(os.path.join(read_dir1, 'segs_1_7_noll_1.fits'))
segs_1_19_noll_1 = fits.getdata(os.path.join(read_dir1, 'segs_1_19_noll_1.fits'))
segs_19_28_noll_1 = fits.getdata(os.path.join(read_dir1, 'segs_19_28_noll_1.fits'))

read_dir2 = '/astro/opticslab1/PASTIS/atlast_data/uncalibrated_analytical_images/2019-04-23-13h-00min_100nm_piston'
segs_1_2_noll_2 = fits.getdata(os.path.join(read_dir2, 'segs_1_2_noll_2.fits'))
segs_1_10_noll_2 = fits.getdata(os.path.join(read_dir2, 'segs_1_10_noll_2.fits'))
segs_1_24_noll_2 = fits.getdata(os.path.join(read_dir2, 'segs_1_24_noll_2.fits'))
segs_1_7_noll_2 = fits.getdata(os.path.join(read_dir2, 'segs_1_7_noll_2.fits'))
segs_1_19_noll_2 = fits.getdata(os.path.join(read_dir2, 'segs_1_19_noll_2.fits'))
segs_19_28_noll_2 = fits.getdata(os.path.join(read_dir2, 'segs_19_28_noll_2.fits'))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(segs_1_2_noll_1, norm=LogNorm(), origin='lower')
plt.title('Piston on segments 1 and 2')
plt.show()

Interestingly, the segments are so close together that our fov is too small to caputre even one fringe!

Moving on to the image iwhere we put **piston** on **segments 19 and 28**. As opposed to the previous image of segments 3 and 11 being aberrated:

1. They are connected by a 45 degree diagonal.
2. They are very far apart, they have the largest possible segment distance on the ATLAST.

This means for the image:
1. The fringes have an orientation in the image that will be 45 degrees tilted the previous one, because the two segment pairs have a different orientation.
2. Since the segments are **far apart**, the fringes in the Fourier plane, here the final image, will be **narrow**.

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(segs_19_28_noll_1, norm=LogNorm(), origin='lower')
plt.title('Piston on segments 19 and 28')
plt.show()

For **piston** on **segments 1 and 24**, we have the same fringe orientation like in the very first image again, because the segment pair has the same orientation in the pupil like the first example before, but because their distance is in between the two previous cases, the fringe spacing will also be somewhere in between.

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(segs_1_24_noll_1, norm=LogNorm(), origin='lower')
plt.title('Piston on segments 1 and 24')
plt.show()

For **piston** on the segment pairs **1-10**, **1-24** and **1-2**, we can see that they have a pair orientation that is the same like in the example above, and since these three pairs have different segment separations, you can see how their fringe spacing differs. In fact, 1-2 is so close that the fringes are so far apart we don't capture them with our field of view.

In [ ]:
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(segs_1_10_noll_1, norm=LogNorm(), origin='lower')
plt.title('Piston on segments 1 and 10')

plt.subplot(1, 3, 2)
plt.imshow(segs_1_24_noll_1, norm=LogNorm(), origin='lower')
plt.title('Piston on segments 1 and 24')

plt.subplot(1, 3, 3)
plt.imshow(segs_1_2_noll_1, norm=LogNorm(), origin='lower')
plt.title('Piston on segments 1 and 2')

plt.show()

#### *Pair redundancy*

See JWST notebook for this.

#### *A different Zernike than piston*

The principles described above will hold true for whatever Zernike we use to aberrate a segment pair. The morphology of the image will change with the chosen Zernike though, as we have see further above when we displayed the different envelopes coming from local Zernikes on the mini segment.

In [ ]:
im_list2 = np.array([segs_1_2_noll_2, segs_1_10_noll_2])
pair_list2 = np.array(['1-2', '1-10'])

plt.figure(figsize=(20, 50))
for i in range(im_list2.shape[0]):
    plt.subplot(4, 2, i+1)
    plt.imshow(im_list2[i], norm=LogNorm(), origin='lower', vmin=1e-10, vmax=1e4)
    plt.title('Segment pair ' + pair_list2[i])

plt.show()

If we compare the pair **19-28** between the **piston** and the **tip** version, we can see that especially the core looks different. And there is that extra dark vertical line in the tip image.

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(1, 2, 1)
plt.imshow(segs_19_28_noll_1, norm=LogNorm(), origin='lower', vmin=1e-10, vmax=1e4)
plt.title('Piston on segments 19 and 28 - Piston')

plt.subplot(1, 2, 2)
plt.imshow(segs_19_28_noll_2, norm=LogNorm(), origin='lower', vmin=1e-10, vmax=1e4)
plt.title('Piston on segments 19 and 28 - Tip')

plt.show()